In [74]:
# coding:utf-8

import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import backend as K
import os
import glob
import matplotlib.gridspec as gridspec
import shutil
from keras.utils import plot_model

In [132]:
def draw_images(datagen, x, result_images):
    # 出力先ディレクトリを作成
    temp_dir = "ll"
    shutil.rmtree(temp_dir)
    os.mkdir(temp_dir)

    # generatorから9個の画像を生成
    # xは1サンプルのみなのでbatch_sizeは1で固定
    g = datagen.flow(x, batch_size=1, save_to_dir=temp_dir, save_prefix='jpg', save_format='jpg')
    g.show()
    for i in range(9):
        batch = g.next()

    # 生成した画像を3x3で描画
    images = glob.glob(os.path.join(temp_dir, "*.jpg"))
    fig = plt.figure()
    gs = gridspec.GridSpec(3, 3)
    gs.update(wspace=0.01, hspace=0.01)
    print(len(images))
    for i in range(9):
        img = load_img(images[i])
        plt.subplot(gs[i])
        plt.imshow(img, aspect='auto')
#         plt.axis("off")
        plt.show()
    plt.savefig(result_images)

    # 出力先ディレクトリを削除
#     shutil.rmtree(temp_dir)

In [140]:
x = img_to_array(load_img('./data/train/carot/0.jpg', target_size=(64,64)))
# print(x)
x = np.expand_dims(x, axis=0)
datagen = ImageDataGenerator(rotation_range=90)
draw_images(datagen, x, './data/train/carot/test.jpg')


AttributeError: 'NumpyArrayIterator' object has no attribute 'show'

In [139]:
# フォルダの中にある画像を順次読み込む
# カテゴリーは0から始める


# 画像の反転と回転の実装、またはdata argumentation の実装を行う
X = []
Y = []

#下、後でコード量を減らす
# 対象Aの画像
for picture in list_pictures('./data/train/carot'):
    img = img_to_array(load_img(picture, target_size=(64,64)))
    X.append(img)
    load_img(picture, target_size=(64,64)).show()
    Y.append(0)


# 対象Bの画像
for picture in list_pictures('./data/train/onion'):
    img = img_to_array(load_img(picture, target_size=(64,64)))
    X.append(img)

    Y.append(1)
    
# 対象Cの画像
for picture in list_pictures('./data/train/potato'):
    img = img_to_array(load_img(picture, target_size=(64,64)))
    X.append(img)

    Y.append(2)

# 対象Dの画像
for picture in list_pictures('./data/train/rice'):
    img = img_to_array(load_img(picture, target_size=(64,64)))
    X.append(img)

    Y.append(3)

# 対象Eの画像
for picture in list_pictures('./data/train/CurryPowder'):
    img = img_to_array(load_img(picture, target_size=(64,64)))
    X.append(img)

    Y.append(4)


# arrayに変換
X = np.asarray(X)
Y = np.asarray(Y)

KeyboardInterrupt: 

In [138]:
# 画素値を0から1の範囲に変換
X = X.astype('float32')
X = X / 255.0

# クラスの形式を変換
Y = np_utils.to_categorical(Y, 5)

# 学習用データとテストデータ
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=111)

AttributeError: 'list' object has no attribute 'astype'

In [50]:
# CNNを構築
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))       # クラスは2個
model.add(Activation('softmax'))

# コンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

# 実行。出力はなしで設定(verbose=0)。
history = model.fit(X_train, y_train, batch_size=5, epochs=30,
                   validation_data = (X_test, y_test), verbose = 1)

# plot_model(model, to_file="model.png", show_shapes=True)
# https://qiita.com/agumon/items/ab2de98a3783e0a93e66

Train on 237 samples, validate on 27 samples
Epoch 1/30

237/237 [==============================] - ETA: 0s - loss: 1.6373 - acc: 0.2025 - val_loss: 1.5532 - val_acc: 0.3704
Epoch 2/30

237/237 [==============================] - ETA: 0s - loss: 1.5893 - acc: 0.2110 - val_loss: 1.5501 - val_acc: 0.4074
Epoch 3/30


KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['acc', 'val_acc'], loc='lower right') #グラフの凡例を用意している
plt.show()

In [34]:
# テストデータに適用
predict_classes = model.predict_classes(X_test)

# マージ。yのデータは元に戻す
mg_df = pd.DataFrame({'predict': predict_classes, 'class': np.argmax(y_test, axis=1)})

# confusion matrix
pd.crosstab(mg_df['class'], mg_df['predict'])


27/27 [==============================] - ETA: 0s


predict,0,1,3,4
class,,,,
0,6,0,2,2
1,0,2,0,0
2,0,3,0,2
3,0,2,4,0
4,0,0,0,4


In [35]:
def train_and_evaluate_model(model,data,labels,data_test,labels_test):
# def train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test]):
    model.fit(data, labels, batch_size=32, epochs=30,verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
    return model.evaluate(data_test, labels_test, verbose=1)  # Evaluate the trained model on the test set!
#損失値と評価の値を算出している
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
# print(cross_val_score(model, image_list, label_list,cv=kfold)).mean()  
from sklearn.cross_validation import StratifiedKFold
label = np.r_[np.repeat(0,16), np.repeat(1,0)]
skf = StratifiedKFold(label, n_folds=15, shuffle=False)
total_score = 0
rep = 0
print("success")
for i, (train, test) in enumerate(skf):
    print("TRAIN:", train, "TEST:", test)
    total_score += train_and_evaluate_model(model, X_train[train], y_train[train], X_train[test], y_train[test])[1]
    rep+=1
# print(total_score/rep)
print("%.10f" % (total_score/rep))

success
TRAIN: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15] TEST: [0 1]
Train on 12 samples, validate on 2 samples
Epoch 1/30

12/12 [==============================] - ETA: 0s - loss: 0.9034 - acc: 0.7500 - val_loss: 0.1055 - val_acc: 1.0000
Epoch 2/30

12/12 [==============================] - ETA: 0s - loss: 0.4204 - acc: 0.8333 - val_loss: 0.1062 - val_acc: 1.0000
Epoch 3/30

12/12 [==============================] - ETA: 0s - loss: 0.2487 - acc: 0.9167 - val_loss: 0.1032 - val_acc: 1.0000
Epoch 4/30

12/12 [==============================] - ETA: 0s - loss: 0.2086 - acc: 0.9167 - val_loss: 0.0864 - val_acc: 1.0000
Epoch 5/30

12/12 [==============================] - ETA: 0s - loss: 0.1650 - acc: 1.0000 - val_loss: 0.0959 - val_acc: 1.0000
Epoch 6/30

12/12 [==============================] - ETA: 0s - loss: 0.1192 - acc: 1.0000 - val_loss: 0.0767 - val_acc: 1.0000
Epoch 7/30

12/12 [==============================] - ETA: 0s - loss: 0.0740 - acc: 1.0000 - val_loss: 0.0695 - val_acc: 1.00

Epoch 2/30

13/13 [==============================] - ETA: 0s - loss: 0.0059 - acc: 1.0000 - val_loss: 0.0522 - val_acc: 1.0000
Epoch 3/30

13/13 [==============================] - ETA: 0s - loss: 0.0082 - acc: 1.0000 - val_loss: 0.0512 - val_acc: 1.0000
Epoch 4/30

13/13 [==============================] - ETA: 0s - loss: 0.0073 - acc: 1.0000 - val_loss: 0.0547 - val_acc: 1.0000
Epoch 5/30

13/13 [==============================] - ETA: 0s - loss: 0.0094 - acc: 1.0000 - val_loss: 0.0517 - val_acc: 1.0000
Epoch 6/30

13/13 [==============================] - ETA: 0s - loss: 0.0623 - acc: 1.0000 - val_loss: 0.0325 - val_acc: 1.0000
Epoch 7/30

13/13 [==============================] - ETA: 0s - loss: 0.0487 - acc: 1.0000 - val_loss: 0.0428 - val_acc: 1.0000
Epoch 8/30

13/13 [==============================] - ETA: 0s - loss: 0.0199 - acc: 1.0000 - val_loss: 0.0373 - val_acc: 1.0000
Epoch 9/30

13/13 [==============================] - ETA: 0s - loss: 0.0345 - acc: 1.0000 - val_loss: 0.0409 - 

Epoch 4/30

13/13 [==============================] - ETA: 0s - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0252 - val_acc: 1.0000
Epoch 5/30

13/13 [==============================] - ETA: 0s - loss: 0.0110 - acc: 1.0000 - val_loss: 0.0213 - val_acc: 1.0000
Epoch 6/30

13/13 [==============================] - ETA: 0s - loss: 0.0294 - acc: 1.0000 - val_loss: 0.0220 - val_acc: 1.0000
Epoch 7/30

13/13 [==============================] - ETA: 0s - loss: 0.0065 - acc: 1.0000 - val_loss: 0.0223 - val_acc: 1.0000
Epoch 8/30

13/13 [==============================] - ETA: 0s - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0233 - val_acc: 1.0000
Epoch 9/30

13/13 [==============================] - ETA: 0s - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0234 - val_acc: 1.0000
Epoch 10/30

13/13 [==============================] - ETA: 0s - loss: 0.0076 - acc: 1.0000 - val_loss: 0.0211 - val_acc: 1.0000
Epoch 11/30

13/13 [==============================] - ETA: 0s - loss: 0.0073 - acc: 1.0000 - val_loss: 0.0199 

Epoch 6/30

13/13 [==============================] - ETA: 0s - loss: 7.1238e-04 - acc: 1.0000 - val_loss: 0.0147 - val_acc: 1.0000
Epoch 7/30

13/13 [==============================] - ETA: 0s - loss: 0.0039 - acc: 1.0000 - val_loss: 0.0163 - val_acc: 1.0000
Epoch 8/30

13/13 [==============================] - ETA: 0s - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0167 - val_acc: 1.0000
Epoch 9/30

13/13 [==============================] - ETA: 0s - loss: 0.0046 - acc: 1.0000 - val_loss: 0.0163 - val_acc: 1.0000
Epoch 10/30

13/13 [==============================] - ETA: 0s - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0161 - val_acc: 1.0000
Epoch 11/30

13/13 [==============================] - ETA: 0s - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0161 - val_acc: 1.0000
Epoch 12/30

13/13 [==============================] - ETA: 0s - loss: 0.0102 - acc: 1.0000 - val_loss: 0.0164 - val_acc: 1.0000
Epoch 13/30

13/13 [==============================] - ETA: 0s - loss: 6.5968e-04 - acc: 1.0000 - val_los

Epoch 8/30

13/13 [==============================] - ETA: 0s - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0218 - val_acc: 1.0000
Epoch 9/30

13/13 [==============================] - ETA: 0s - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0199 - val_acc: 1.0000
Epoch 10/30

13/13 [==============================] - ETA: 0s - loss: 0.0024 - acc: 1.0000 - val_loss: 0.0185 - val_acc: 1.0000
Epoch 11/30

13/13 [==============================] - ETA: 0s - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0185 - val_acc: 1.0000
Epoch 12/30

13/13 [==============================] - ETA: 0s - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0175 - val_acc: 1.0000
Epoch 13/30

13/13 [==============================] - ETA: 0s - loss: 0.0042 - acc: 1.0000 - val_loss: 0.0168 - val_acc: 1.0000
Epoch 14/30

13/13 [==============================] - ETA: 0s - loss: 4.9442e-04 - acc: 1.0000 - val_loss: 0.0169 - val_acc: 1.0000
Epoch 15/30

13/13 [==============================] - ETA: 0s - loss: 0.0010 - acc: 1.0000 - val_loss:


13/13 [==============================] - ETA: 0s - loss: 6.7948e-04 - acc: 1.0000 - val_loss: 0.0062 - val_acc: 1.0000
Epoch 10/30

13/13 [==============================] - ETA: 0s - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0066 - val_acc: 1.0000
Epoch 11/30

13/13 [==============================] - ETA: 0s - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0066 - val_acc: 1.0000
Epoch 12/30

13/13 [==============================] - ETA: 0s - loss: 6.4474e-04 - acc: 1.0000 - val_loss: 0.0067 - val_acc: 1.0000
Epoch 13/30

13/13 [==============================] - ETA: 0s - loss: 0.0055 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 14/30

13/13 [==============================] - ETA: 0s - loss: 0.0042 - acc: 1.0000 - val_loss: 0.0037 - val_acc: 1.0000
Epoch 15/30

13/13 [==============================] - ETA: 0s - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 16/30

13/13 [==============================] - ETA: 0s - loss: 0.0037 - acc: 1.0000 - val_loss: 0.004

Epoch 10/30

13/13 [==============================] - ETA: 0s - loss: 6.5870e-04 - acc: 1.0000 - val_loss: 0.0102 - val_acc: 1.0000
Epoch 11/30

13/13 [==============================] - ETA: 0s - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0096 - val_acc: 1.0000
Epoch 12/30

13/13 [==============================] - ETA: 0s - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0096 - val_acc: 1.0000
Epoch 13/30

13/13 [==============================] - ETA: 0s - loss: 0.0030 - acc: 1.0000 - val_loss: 0.0091 - val_acc: 1.0000
Epoch 14/30

13/13 [==============================] - ETA: 0s - loss: 5.6059e-04 - acc: 1.0000 - val_loss: 0.0091 - val_acc: 1.0000
Epoch 15/30

13/13 [==============================] - ETA: 0s - loss: 2.6280e-04 - acc: 1.0000 - val_loss: 0.0091 - val_acc: 1.0000
Epoch 16/30

13/13 [==============================] - ETA: 0s - loss: 3.9206e-04 - acc: 1.0000 - val_loss: 0.0091 - val_acc: 1.0000
Epoch 17/30

13/13 [==============================] - ETA: 0s - loss: 0.0055 - acc: 1.00

Epoch 10/30

13/13 [==============================] - ETA: 0s - loss: 8.8163e-04 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 11/30

13/13 [==============================] - ETA: 0s - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 12/30

13/13 [==============================] - ETA: 0s - loss: 6.8595e-04 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 13/30

13/13 [==============================] - ETA: 0s - loss: 0.0056 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 1.0000
Epoch 14/30

13/13 [==============================] - ETA: 0s - loss: 7.1014e-04 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 1.0000
Epoch 15/30

13/13 [==============================] - ETA: 0s - loss: 2.4143e-04 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 1.0000
Epoch 16/30

13/13 [==============================] - ETA: 0s - loss: 0.0058 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 17/30

13/13 [==============================] - ETA: 0s - loss: 0.0021 - acc: 1.00